<a href="https://colab.research.google.com/github/genfuru011/AD_Tech_SLM/blob/main/AD_Tech_SLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AD_Tech_SLM
広告特化型 SLM（Small Language Model）開発・運用プロジェクト

## プロジェクト概要

本プロジェクトは、“広告コピーの品質向上”と“CTR（クリック率）改善”を目指す、広告業界向けの特化型言語モデルを開発・運用するものです。  
データ収集からモデル開発・評価・デプロイ・運用まで、一貫したワークフローを設計し、効率的かつ安定的な価値提供を目指します。

---

## ロードマップ

### 短期（2ヶ月）：PoC → 本格トレーニング
### 中期（4ヶ月）：デプロイ＋運用パイプライン構築
### 長期（6ヶ月以降）：継続的改善・新機能展開

---

## チューニング手法

### 人間のフィードバックを活用（RLHF, DPO手法）
- 人間のフィードバックをもとに、より最適なコピーを生成するようモデルを学習させる
- 感情制御、要約生成、対話タスクなど、様々なタスクでRLHFと同等かそれ以上の人間選好一致性を示すことが報告されています。特に、トーンやスタイル、特定のコンテンツ指向など、主観的な要素が重要となる場合に効果を発揮します。
- 特に、トーンやスタイル、特定のコンテンツ指向など、主観的な要素が重要となる場合に効果を発揮します。
- 人間が好む出力と好まない出力のペアデータが不可欠です。このデータの質や量、多様性がモデルの性能に大きく影響します

### 必要データアセットJSONL
```json
{"prompt": "【テーマ】雨の日でもワクワクするニュースアプリを紹介してください",
 "chosen": "雨が降っても最新トレンドをスマホでサクッとチェック！天気と話題を同時にキャッチして、移動中も退屈知らず♪",
 "rejected": "ニュースが見られるよ！便利！"}

{"prompt": "【テーマ】忙しい会社員が移動中に英語を学べるアプリを紹介してください",
 "chosen": "通勤電車で 3 分！AI レッスンがあなたの発音を即フィードバック💡スキマ時間で着実にスキルアップ！",
 "rejected": "英語学習ができます。おすすめです。"}

{"prompt": "【テーマ】節電をサポートする家計簿アプリを紹介してください",
 "chosen": "家電ごとの電気代を自動計算！グラフでムダを一目で発見して、月末の請求額にもうドキドキしません✨",
 "rejected": "節約に役立つアプリです。ぜひ使ってください。"}
```
- prompt：コピーを書かせたいお題や前提条件をまとめます。
- chosen：CTR が高かった “良い広告テキスト” を入れます。
- rejected：CTR が低かった、または不採用になった文を入れます。

# settings command


## **ライブラリのインストール**

# =========================================
# 1. ライブラリのインストール
# -----------------------------------------

In [1]:
import torch, platform, os, subprocess, textwrap, json, sys
print("Python   :", platform.python_version())
print("CUDA     :", torch.version.cuda)
!nvidia-smi -L    # A100 40GB であることを確認


Python   : 3.11.12
CUDA     : 12.4
GPU 0: NVIDIA A100-SXM4-40GB (UUID: GPU-977f29ad-b600-d33c-53b9-29903476cc47)


1. パッケージを最新版に揃える
bash
コピーする
編集する


In [2]:
!pip install -q --upgrade trl accelerate transformers peft bitsandbytes datasets sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2. Hugging Face 認証（Hameln は利用制限付きモデル）
python
コピーする
編集する


In [3]:
from huggingface_hub import login
login()        # → アクセストークンをペースト


3. 4-bit + LoRA + DPO のコンフィグ作成
python
コピーする
編集する


In [4]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

base_id = "Elizezen/Hameln-japanese-mistral-7B"

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

tok    = AutoTokenizer.from_pretrained(base_id, use_fast=False)
tok.pad_token = tok.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    base_id,
    quantization_config=bnb_cfg,
    device_map="auto",
    trust_remote_code=True,
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

4 LoRA 設定（dict 形式にしておく）
python
コピーする
編集する


In [5]:
# 4️⃣ LoRA 設定（インスタンスのまま保持）
from peft import LoraConfig
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj",
                    "gate_proj","up_proj","down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

データセット読み込み
python
コピーする
編集する


In [6]:
from datasets import load_dataset
train_ds = load_dataset("shi3z/anthropic_hh_rlhf_japanese", split="train")


README.md:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

arlhfj.jsonl:   0%|          | 0.00/371M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

DPOConfig（学習用パラメータのみ！）

In [9]:
from trl import DPOConfig

# 6️⃣ DPOConfig ― 変更点だけ抜粋
dpo_cfg = DPOConfig(
    output_dir="hameln_dpo_lora",
    per_device_train_batch_size = 1,     # ↓ 4→1
    gradient_accumulation_steps = 8,     # 実バッチは 8 のまま
    gradient_checkpointing      = True,  # ★ 追加
    num_train_epochs            = 1,
    learning_rate               = 2e-4,
    beta                        = 0.1,
    logging_steps               = 50,
    save_steps                  = 200,
    eval_steps                  = 200,
    bf16                        = True,
)


DPOTrainer 生成 → 学習開始
python
コピーする
編集する


In [11]:
# 7️⃣ DPOTrainer 生成
from trl import DPOTrainer

trainer = DPOTrainer(
    model            = base_model,     # 4-bit 量子化済み Hameln
    ref_model        = None,           # デフォルトでコピーを作成
    args             = dpo_cfg,        # ← DPOConfig（学習パラメータのみ）
    processing_class = tok,            # トークナイザ
    train_dataset    = train_ds,       # 161k 行の比較データ
    peft_config      = lora_cfg,       # ← 辞書ではなく **LoraConfig**
)
trainer.train()


OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 200.88 MiB is free. Process 8373 has 39.35 GiB memory in use. Of the allocated memory 37.79 GiB is allocated by PyTorch, and 1.07 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)